In [1]:
import pandas as pd

In [2]:
import os
import sys

os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-21'
#os.environ['PYSPARK_HOME'] = r'C:\Users\lpaschoal\pyspark\spark-4.0.1-bin-hadoop3'
os.environ['SPARK_HOME'] = r'C:\Users\lpaschoal\pyspark\spark-4.0.1-bin-hadoop3'
os.environ['HADOOP_HOME'] = r'C:\Users\lpaschoal\pyspark\hadoop\winutils\hadoop-3.3.6'

sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\pyspark")
sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\lib\pyspark.zip")
sys.path.insert(0, os.getenv('SPARK_HOME') + r"\python\lib\py4j-0.10.9.9-src.zip")

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,SelectedColumn

spark = SparkSession.builder.appName("Coalesce and Repartition")\
    .config("spark.driver.memory", "12g") \
    .config("spark.driver.extraJavaOptions", "--add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED") \
    .config("spark.executor.extraJavaOptions", "--add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.pyspark.udf.faulthandler.enabled", "true")

In [3]:
#Reading CSVs
#cards_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\cards_data.csv",header=True, inferSchema=True).createOrReplaceTempView('cards_data')
#trans_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\transactions_data.csv",header=True, inferSchema=True).createOrReplaceTempView('trans_data')
#users_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\users_data.csv",header=True, inferSchema=True).createOrReplaceTempView('users_data')

cards_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\cards_data.csv",header=True, inferSchema=True)
trans_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\transactions_data.csv",header=True, inferSchema=True)
users_data = spark.read.csv(r"C:\Users\lpaschoal\data_src\financial_data\users_data.csv",header=True, inferSchema=True)

In [4]:
cards_data.show()

+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|  id|client_id|card_brand|      card_type|     card_number|expires|cvv|has_chip|num_cards_issued|credit_limit|acct_open_date|year_pin_last_changed|card_on_dark_web|
+----+---------+----------+---------------+----------------+-------+---+--------+----------------+------------+--------------+---------------------+----------------+
|4524|      825|      Visa|          Debit|4344676511950444|12/2022|623|     YES|               2|      $24295|       09/2002|                 2008|              No|
|2731|      825|      Visa|          Debit|4956965974959986|12/2020|393|     YES|               2|      $21968|       04/2014|                 2014|              No|
|3701|      825|      Visa|          Debit|4582313478255491|02/2024|719|     YES|               2|      $46414|       07/2003|                 2004|              No|
|  4

### Checking if there is skewed data

In [5]:
from pyspark.sql.functions import col, count, desc
import plotly.express as px
import random
import pandas as pd

df = cards_data
category_counts = df.groupBy("card_brand").count().orderBy(desc("count"))
category_counts.show()

# Convert to Pandas for plotting
pandas_df = category_counts.toPandas()

# Create bar chart
fig = px.bar(
    pandas_df, 
    x='card_brand', 
    y='count',
    title='Card Type Distribution',
    color='count',
    color_continuous_scale='Viridis',
    text='count'  # Show numbers on bars
)

fig.update_layout(
    xaxis_title="Card Type",
    yaxis_title="Count",
    width=800,
    height=500
)
fig.show()

+----------+-----+
|card_brand|count|
+----------+-----+
|Mastercard| 3209|
|      Visa| 2326|
|      Amex|  402|
|  Discover|  209|
+----------+-----+



In [25]:
cards_data = cards_data.repartition(4,'card_brand')

In [26]:
cards_data.rdd.getNumPartitions()

4